In [72]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
import glob
import pathlib
from sklearn.model_selection import train_test_split

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


# Data preprocessing

In [4]:
tf.__version__

'2.5.0'

In [84]:
image_input = sorted(tf.io.gfile.glob('./ISIC2018_Task1-2_Training_Input_x2/*.jpg'))

In [85]:
len(image_input)

2594

In [86]:
ground_truth = sorted(tf.io.gfile.glob('./ISIC2018_Task1_Training_GroundTruth_x2/*.png'))

In [87]:
len(ground_truth)

2594

In [88]:
image_input[:5]

['.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000000.jpg',
 '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000001.jpg',
 '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000003.jpg',
 '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000004.jpg',
 '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0000006.jpg']

In [89]:
ground_truth[:5]

['.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000000_segmentation.png',
 '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000001_segmentation.png',
 '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000003_segmentation.png',
 '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000004_segmentation.png',
 '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0000006_segmentation.png']

In [90]:
# Disorder the data and keep the input image and truth ground one-to-one correspondence
index = np.random.permutation(len(image_input))
image_input = np.array(image_input)[index]
ground_truth = np.array(ground_truth)[index]

In [91]:
image_input[:5]

array(['.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0013136.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0015568.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0010038.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0014891.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0014683.jpg'],
      dtype='<U53')

In [92]:
ground_truth[:5]

array(['.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0013136_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0015568_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0010038_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0014891_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0014683_segmentation.png'],
      dtype='<U70')

In [93]:
length = len(image_input)
print(length)

2594


In [116]:
image_input_val = image_input[:(int(length*0.2))]
print(len(image_input_val))
ground_truth_val = ground_truth[:(int(length*0.2))]
print(len(ground_truth_val))

518
518


In [105]:
image_input_val[:5]

array(['.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0013136.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0015568.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0010038.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0014891.jpg',
       '.\\ISIC2018_Task1-2_Training_Input_x2\\ISIC_0014683.jpg'],
      dtype='<U53')

In [106]:
ground_truth_val[:5]

array(['.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0013136_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0015568_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0010038_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0014891_segmentation.png',
       '.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0014683_segmentation.png'],
      dtype='<U70')

In [118]:
image_input_test = image_input[int(length*0.2):int(length*0.4)]
ground_truth_test = ground_truth[int(length*0.2):int(length*0.4)]

In [119]:
image_input_train = image_input[int(length*0.4):]
ground_truth_train = ground_truth[int(length*0.4):]

In [120]:
print(len(image_input_train))
print(len(ground_truth_train))

1557
1557


In [109]:
ground_truth_val[-1]

'.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0014868_segmentation.png'

In [110]:
ground_truth_train[0]

'.\\ISIC2018_Task1_Training_GroundTruth_x2\\ISIC_0014962_segmentation.png'

In [111]:
train_ds = tf.data.Dataset.from_tensor_slices((image_input_train, ground_truth_train))

In [112]:
train_ds

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.string)>

In [113]:
val_ds = tf.data.Dataset.from_tensor_slices((image_input_val, ground_truth_val))

In [121]:
test_ds = tf.data.Dataset.from_tensor_slices((image_input_test, ground_truth_test))

In [114]:
def read_jpg(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(imag, channel=3)
    return image

In [115]:
def read_png(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1)
    return image